# 4. Analysis

- premiers tests à l'arrache

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/df_repu.csv")

In [3]:
df.shape

(4592, 61)

## décider de comment on découpe
- sentence ?
- paragraphe (existe pas ici)
- dire que tant pis ?
- faire un check global du nb token par intervention et cut que ce qui dépasse ?
- etc.

## Des tests à l'arrache

In [44]:
df["DateSeance_ts"] = pd.to_datetime(df["DateSeance"], format="%Y%m%d%H%M%S%f")
df["DateSeance_day"] = df["DateSeance_ts"].dt.date

0       2023-03-13
1       2023-03-13
2       2023-03-13
3       2023-03-13
4       2023-03-13
           ...    
4587    2023-02-10
4588    2023-02-10
4589    2023-02-10
4590    2023-02-27
4591    2023-02-27
Name: DateSeance_day, Length: 4592, dtype: object

In [4]:
# nettoyage moche du texte
# sinon à gérer avant !

import re


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)  # (...)
    # texte = re.sub(r"\[[^\]]*\]", "", texte)     # [...] # TODO: vérif avant
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

In [5]:
df["Texte_clean"].str.len().describe()

count     4592.000000
mean      1582.548780
std       1773.677754
min         25.000000
25%        475.000000
50%       1017.500000
75%       1983.000000
max      20908.000000
Name: Texte_clean, dtype: float64

In [6]:
# # tester un truc autour de alibaba pour l'embedding ?

# import pandas as pd
# from sentence_transformers import SentenceTransformer
# import numpy as np

# # Prepare embedding model
# model_name_or_path="Alibaba-NLP/gte-multilingual-base"
# model = SentenceTransformer(model_name_or_path, trust_remote_code=True)

# # Embed the texts
# embeddings = [model.encode(x, normalize_embeddings=True) for x in sotu_sample["paragraph"].to_list()]
# embeddings = np.array(embeddings)

In [12]:
# si besoin vérif usage de bancs dans les textes (hors des didascalies)
bancs = df[df["Texte_clean"].str.contains("bancs")]
bancs

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj,repu_match_valide,repu_match_simple,Texte_clean
2,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,7 ans,0.17,0.20,0.977,0.216,0.0,2025-06-24,True,True,"C’est notre sentiment, celui des élus sur les ..."
42,CRSANR5L16S2024O1N117,RUANR5L16S2024IDS28058,SCR5A2024O1,20240206150000000,mardi 06 février 2024,1,117,AN,16,Session ordinaire 2023-2024,...,2 ans,0.17,0.23,0.990,0.245,0.0,2025-06-24,True,True,"…Julien Luya, Robert Siegel ou, hier encore, l..."
44,CRSANR5L16S2024O1N117,RUANR5L16S2024IDS28058,SCR5A2024O1,20240206150000000,mardi 06 février 2024,1,117,AN,16,Session ordinaire 2023-2024,...,3 ans,0.32,0.49,0.985,0.000,1.0,2025-06-24,True,True,"Tout à l’heure, sur les bancs de l’extrême gau..."
51,CRSANR5L16S2024O1N103,RUANR5L16S2024IDS28011,SCR5A2024O1,20240124140000000,mercredi 24 janvier 2024,1,103,AN,16,Session ordinaire 2023-2024,...,7 ans,0.12,0.33,0.984,0.000,0.0,2025-06-24,True,False,"L’histoire de l’avortement, c’est l’histoire d..."
54,CRSANR5L16S2024O1N103,RUANR5L16S2024IDS28011,SCR5A2024O1,20240124140000000,mercredi 24 janvier 2024,1,103,AN,16,Session ordinaire 2023-2024,...,8 ans,0.16,0.37,0.952,0.000,1.0,2025-06-24,True,True,« Les représentants du peuple français […] ont...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476,CRSANR5L16S2024O1N046,RUANR5L16S2024IDS27703,SCR5A2024O1,20231108150000000,mercredi 08 novembre 2023,1,46,AN,16,Session ordinaire 2023-2024,...,7 ans,0.14,0.37,0.984,0.217,0.0,2025-06-24,True,True,…éludant le sujet de l’imposition des sociétés...
4478,CRSANR5L16S2024O1N046,RUANR5L16S2024IDS27703,SCR5A2024O1,20231108150000000,mercredi 08 novembre 2023,1,46,AN,16,Session ordinaire 2023-2024,...,8 ans,0.25,0.25,0.968,0.000,1.0,2025-06-24,True,True,Je remercie le rapporteur général et le minist...
4498,CRSANR5L16S2024O1N091,RUANR5L16S2024IDS28041,SCR5A2024O1,20231221153000000,jeudi 21 décembre 2023,Unique,91,AN,16,Session ordinaire 2023-2024,...,3 ans,0.30,0.41,0.974,0.000,1.0,2025-06-24,True,True,…l’essentiel étant de ne jamais perdre. Élisab...
4557,CRSANR5L16S2023O1N220,RUANR5L16S2023IDS27020,SCR5A2023O1,20230503150000000,mercredi 03 mai 2023,1,220,AN,16,Session ordinaire 2022-2023,...,3 ans,0.07,0.13,0.933,0.000,1.0,2025-06-24,True,False,"Enfin, je veux réaffirmer que nous soutenons l..."


https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html

In [16]:
# regarder pour faire autre chose que nltk, etc.

import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import bertopic

nltk.download("stopwords")
french_stopwords = list(set(stopwords.words("french")))
vectorizer_model = CountVectorizer(stop_words=french_stopwords)
topic_model = bertopic.BERTopic(language="french", vectorizer_model=vectorizer_model)
# topics, probs = topic_model.fit_transform(df["Texte_clean"])

[nltk_data] Downloading package stopwords to /Users/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# regarder pour faire autre chose que nltk, etc.

# from bertopic import BERTopic
# topic_model = BERTopic(language="french", min_topic_size=35, verbose=True)
# topics, _ = topic_model.fit_transform(df['Texte_clean'])

In [17]:
topics, probs = topic_model.fit_transform(df["Texte_clean"])

In [18]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1982,-1_plus_république_cette_loi,"[plus, république, cette, loi, comme, tout, êt...","[Les associations agréées de sécurité civile, ..."
1,0,369,0_école_élèves_éducation_enfants,"[école, élèves, éducation, enfants, enseignant...","[Depuis 2017, le Gouvernement a placé l’éducat..."
2,1,239,1_procureur_justice_république_douaniers,"[procureur, justice, république, douaniers, am...",[…que confirmeraient les chiffres de la récidi...
3,2,142,2_france_français_république_plus,"[france, français, république, plus, tous, pay...",[« La France est une République indivisible » ...
4,3,125,3_immigration_étrangers_asile_droit,"[immigration, étrangers, asile, droit, france,...",[La proposition de loi qui nous est soumise au...
...,...,...,...,...,...
59,58,11,58_mots_républicains_préfectoral_confiance,"[mots, républicains, préfectoral, confiance, a...",[Je partage une part des convictions républica...
60,59,11,59_jeux_sport_olympiques_paralympiques,"[jeux, sport, olympiques, paralympiques, événe...","[En effet, la deuxième édition de la Journée p..."
61,60,11,60_élus_leurs_violence_agressions,"[élus, leurs, violence, agressions, violences,...",[Ces attaques de grande ampleur s’ajoutent aux...
62,61,10,61_tarifs_énergies_renouvelables_électricité,"[tarifs, énergies, renouvelables, électricité,...",[L’amendement no 812 concerne la Nouvelle-Calé...


In [39]:
fig_bert_moche = topic_model.visualize_topics()
fig_bert_moche

In [40]:
fig_bert_moche.write_html("fig_bert_moche.html")

In [41]:
bert_moche = topic_model.get_topic_info()
bert_moche.to_csv("../data/intermediate/bert_moche.csv")

In [ ]:
topic_model.save("../models/bert/modele_bert_moche", save_embedding_model=True)

2025-06-29 23:19:18,414 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


## Dynamic topic model

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20.


In [54]:
# Idée de base :
# topics_over_time = model.topics_over_time(documents, timestamps)
# model.visualize_topics_over_time(topics_over_time)

topics_over_time = topic_model.topics_over_time(
    docs=df["Texte_clean"],
    timestamps=df["DateSeance_ts"],
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=20,
)

In [55]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## topic reduction

In [56]:
# topic_model.reduce_topics(df["Texte_clean"], nr_topics=20)

In [57]:
# topic_model.get_topic_info()

In [53]:
# topics = topic_model.topics_
# topics

## Bert things

In [32]:
# import nltk
# from nltk.corpus import stopwords
# from sentence_transformers import SentenceTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# import bertopic

# nltk.download("stopwords")
# french_stopwords = list(set(stopwords.words("french")))
# vectorizer_model = CountVectorizer(stop_words=french_stopwords)
# topic_model = bertopic.BERTopic(language="french", vectorizer_model=vectorizer_model)
# topics, probs = topic_model.fit_transform(df["Texte_clean"])

In [ ]:
# TODO: redo with the good sentencetransformer

Stopwords :
- https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#document-length
- Instead, we can use the CountVectorizer to preprocess our documents after having generated embeddings and clustered our documents. Personally, I have found almost no disadvantages to using the CountVectorizer to remove stopwords and it is something I would strongly advise to try out:
- We can also use the ClassTfidfTransformer to reduce the impact of frequent words. The end result is very similar to explicitly removing stopwords but this process does this automatically:

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import spacy
from bertopic.vectorizers import ClassTfidfTransformer

# import nltk
# from nltk.corpus import stopwords

# vectorizer_model et french stopwords
# Avec spacy
nlp = spacy.load("fr_core_news_sm")
french_stopwords = list(nlp.Defaults.stop_words)
vectorizer_model = CountVectorizer(stop_words=french_stopwords)
# # Ou avec nltk
# nltk.download("stopwords")
# french_stopwords = list(set(stopwords.words("french")))
# vectorizer_model = CountVectorizer(stop_words=french_stopwords)

# "dangvantuan/sentence-camembert-large"
# "Alibaba-NLP/gte-multilingual-base"
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# créer le modèle
topic_model_better = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,  # remove stopwords after embbedings
    # ctfidf_model=ctfidf_model, # (or) reduce the impact of frequent word
)

# Fiter le modèle
topics_better, probs_better = topic_model_better.fit_transform(df["Texte_clean"])


In [ ]:
# model = BERTopic(
#     embedding_model=...,         # Modèle pour transformer les textes en vecteurs
#     top_n_words=10,              # Nombre de mots clés par sujet
#     nr_topics="auto",            # Réduction automatique du nombre de sujets
#     calculate_probabilities=True,# Calcul de la probabilité d'appartenance à chaque sujet
#     verbose=True,                # Affichage des logs
#     min_topic_size=10,           # Taille minimale d’un cluster pour former un sujet
#     language="english",          # Langue utilisée pour les stopwords (uniquement si vectorizer_model est défini)
#     vectorizer_model=...,        # Modèle TF-IDF custom
#     ctfidf_model=...,            # Class-based TF-IDF (CTFIDF) custom
#     diversity=0.5,               # Diversité des mots-clés (0 = mots fréquents, 1 = mots uniques)
#     seed_topic_list=...,         # Thèmes pré-définis
# )

In [72]:
# model_better?

In [82]:
# Step 6: Review topics
topic_info_better = topic_model_better.get_topic_info()
topic_info_better


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2554,-1_qu_république_loi_été,"[qu, république, loi, été, bien, faire, france...",[Permettez-moi tout d’abord de saluer l’excell...
1,0,129,0_mer_territoires_prix_climatique,"[mer, territoires, prix, climatique, loi, qu, ...","[Il y a cinquante ans, le commandant Cousteau ..."
2,1,121,1_femmes_droits_liberté_droit,"[femmes, droits, liberté, droit, discriminatio...","[La République s’incarne à travers ce texte, e..."
3,2,112,2_violence_élus_violences_extrême,"[violence, élus, violences, extrême, droite, t...",[Je vous remercie de m’avoir invité pour débat...
4,3,96,3_république_ve_démocratie_abandonne,"[république, ve, démocratie, abandonne, vive, ...","[Le plus jeune de toute la Ve République !, Vo..."
...,...,...,...,...,...
60,59,12,59_langues_langue_régionales_française,"[langues, langue, régionales, française, ensei...",[La langue française est un véritable joyau du...
61,60,12,60_polynésie_ordonnance_fonction_publique,"[polynésie, ordonnance, fonction, publique, fr...","[Permettez-moi, en tant qu’ancien Montalbanais..."
62,61,12,61_arménie_karabakh_azerbaïdjan_arménien,"[arménie, karabakh, azerbaïdjan, arménien, arm...","[Le 9 avril 1916, alors que le gouvernement de..."
63,62,11,62_finances_dette_entreprises_groupe,"[finances, dette, entreprises, groupe, projet,...",[La proposition de loi visant à faciliter le p...


In [ ]:
# Optional: visualize
topic_model_better.visualize_topics().show()

In [ ]:
topic_model_better.save("../models/bert/modele_bert_better", save_embedding_model=True)

2025-06-30 00:25:23,641 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [91]:
topics_over_time_better = topic_model_better.topics_over_time(
    docs=df["Texte_clean"],
    timestamps=df["DateSeance_ts"],
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=20,
)

In [93]:
topic_model_better.visualize_topics_over_time(topics_over_time_better, top_n_topics=40)

In [95]:
df["DateSeance_ts"].sort_values()

1448   2021-02-01 16:00:00
1412   2021-02-01 16:00:00
1411   2021-02-01 16:00:00
1410   2021-02-01 16:00:00
1409   2021-02-01 16:00:00
               ...        
2927   2024-06-06 15:00:00
3257   2024-06-06 21:45:00
3256   2024-06-06 21:45:00
3357   2024-06-07 10:00:00
2673   2024-06-07 15:00:00
Name: DateSeance_ts, Length: 4592, dtype: datetime64[ns]

In [98]:
df["DateSeance_day"].sort_values()

1452    2021-02-01
1416    2021-02-01
1415    2021-02-01
1414    2021-02-01
1413    2021-02-01
           ...    
3256    2024-06-06
3257    2024-06-06
2926    2024-06-06
2673    2024-06-07
3357    2024-06-07
Name: DateSeance_day, Length: 4592, dtype: object

In [101]:
temp_date = df[df["DateSeance_day"] < pd.to_datetime("2022-01-01").date()]
temp_date

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj,repu_match_valide,repu_match_simple,Texte_clean,DateSeance_ts,DateSeance_day
1395,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,NaN,0.912,0.000,0.0,2025-06-24,True,False,"Monsieur le président, mesdames, messieurs les...",2021-02-01 16:00:00,2021-02-01
1396,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,NaN,0.912,0.000,0.0,2025-06-24,True,False,"…mais un texte pour tout le monde, qui aura de...",2021-02-01 16:00:00,2021-02-01
1397,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,NaN,0.912,0.000,0.0,2025-06-24,True,False,Cette précision prouve sans doute que les révo...,2021-02-01 16:00:00,2021-02-01
1398,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,NaN,0.912,0.000,0.0,2025-06-24,True,False,Des puissances étrangères n’ont pas à payer de...,2021-02-01 16:00:00,2021-02-01
1399,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,NaN,0.912,0.000,0.0,2025-06-24,True,True,…nous considérons que toutes les religions doi...,2021-02-01 16:00:00,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,0.47,0.974,0.000,1.0,2025-06-24,True,True,"Collègue Mélenchon, le groupe Dem n’associera ...",2021-02-01 16:00:00,2021-02-01
1456,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,0.25,0.968,0.000,1.0,2025-06-24,True,False,"Depuis des mois, de discours solennels en entr...",2021-02-01 16:00:00,2021-02-01
1457,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,0.25,0.968,0.000,1.0,2025-06-24,True,True,"…une loi qui prétend, de surcroît, se hisser à...",2021-02-01 16:00:00,2021-02-01
1458,CRSANR5L16S2021O1N144,RUANR5L15S2021IDS23666,SCR5A2021O1,20210201160000000,lundi 01 février 2021,1,144,AN,16,session ordinaire 2020-2021,...,0.27,0.969,0.689,0.0,2025-06-24,True,True,Ce projet de loi visant à renforcer les princi...,2021-02-01 16:00:00,2021-02-01


In [103]:
temp_date["UID"].unique()

array(['CRSANR5L16S2021O1N144'], dtype=object)

In [ ]:
df["Session"].value_counts(dropna=False)

NameError: name 'df' is not defined

voir le modèle dans le notebook emilien qui découpe en paragraphe sans avoir de marqueur de paragrphae. et garder que celui qui contient répu.

Passer à l'échelle de la phrase pour tout ce qui va être pour sentiment, réseau de mots, proba des termes, etc.